<a href="https://colab.research.google.com/github/paolobiancalana/crypto-trading-bot-rl/blob/main/Crypto_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1 - Install Dependencies

In [ ]:
!pip install alpaca-py
!pip install -q alpaca-trade-api
!pip install pycoingecko
!pip install requests
!pip install openai
!pip3 install git+https://github.com/JustAnotherArchivist/snscrape.git
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.8/96.8 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.7/323.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 20.9 MB/s eta 0:00:00
Reason for being yanked: This version includes overly restrictive multidict upper boundary disallowing multidict v6+. The previous patch version didn't have that and this is now causing dependency resolution problems for the users who have an "incompatible" version pinned. This is not really necessary anymore and will be addressed in the next release v3.8.3

https://github.com/aio-libs/aiohttp/pull/6950
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-zrpxrwq6
  Running command

# Step 2 - Import Data

In [ ]:
import numpy as np
from pycoingecko import CoinGeckoAPI
import requests
import snscrape.modules.twitter as sntwitter
import pandas as pd
import textwrap
import datetime as dt
import openai
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
import re
import time
import pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


### Data Source 1/4: Price History

In [ ]:
cg = CoinGeckoAPI()

cryptos = ['chainlink', 'ethereum', 'matic-network', 'solana', 'presearch']

crypto_data = ''

for crypto in cryptos:
    data = cg.get_coin_market_chart_by_id(crypto, vs_currency='usd', days=30)
    prices = data['prices']
    prices_only = [price[1] for price in prices]
    high = max(prices_only)
    low = min(prices_only)
    avg = np.mean(prices_only)
    crypto_data+=f" {crypto} data for the past 30 days: High={high}, Low={low}, Average={avg}"
    print(f"{crypto} data for the past 30 days: High={high}, Low={low}, Average={avg}")

bitcoin data for the past 30 days: High=31185.219831413775, Low=24864.608901885007, Average=27555.093029453637
ethereum data for the past 30 days: High=1922.831648666678, Low=1632.457370596918, Average=1818.1107357196472
matic-network data for the past 30 days: High=0.9087424514904233, Low=0.5579180724197483, Average=0.7090857588377285
solana data for the past 30 days: High=22.23048506493996, Low=13.805519170106006, Average=17.48240568534618
avalanche-2 data for the past 30 days: High=14.905311045572896, Low=10.754774467001235, Average=12.90989605113597


### Data Source 3/4: News Sentiment

In [ ]:
symbols = ["LINK", "ETH", "PRE", "SOL", "PRE"]

def get_all_crypto_news():
    API_KEY = "ENTER YOUR KEY"
    all_news = {}

    for symbol in symbols:
        url = f'https://newsapi.org/v2/everything?q={symbol}&apiKey={API_KEY}'
        response = requests.get(url)
        data = response.json()

        news_data = []
        try:
            for article in data['articles'][:3]:  # Limit to top 3 articles
                news_data.append({
                    'title': article['title'],
                    'source': article['source']['name'],
                })
            all_news[symbol] = news_data
        except:
            return all_news

    return all_news

news_output = get_all_crypto_news()
print(textwrap.fill(str(news_output), width=50))

{'BTC': [{'title': 'DOJ charges Russian nationals
with laundering bitcoin in 2011 Mt. Gox hack',
'source': 'Engadget'}, {'title': 'Altcoins:
Exploring the Best Cryptocurrencies Beyond BTC',
'source': 'ReadWrite'}, {'title': '9 Years After
the Mt. Gox Hack, Feds Indict Alleged Culprits',
'source': 'Wired'}], 'ETH': [{'title': 'Apollo
Back end just made public', 'source': 'Reddit
/r/all'}, {'title': 'Los dentistas se han topado
con un enemigo inesperado: los propios dentistas y
los tratamientos innecesarios', 'source':
'Xataka.com'}, {'title': 'The mystery of our
transparent universe has been solved!', 'source':
'Yahoo Entertainment'}], 'MATIC': [{'title':
'Coinbase and Binance Lawsuits Put Crypto on Ice',
'source': 'Wired'}, {'title': 'Dogecoin and
Polygon Price Prediction and Beyond: a Look at Key
Altcoins for 2023', 'source': 'Due.com'},
{'title': 'Robinhood App Will End Support for
Three Cryptocurrency Tokens After June 27',
'source': 'Slashdot.org'}], 'SOL': [{'title': 'Sol
Reader I

### Data Source 2/4: Twitter Sentiment


In [ ]:
queries = ["LINK", "ETH", "PRE", "SOL", "LTC"]
tweets_list = []

for query in queries:
    num = 0
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        if num == 10:
            break
        num += 1
        tweets_list.append(tweet.rawContent)
        print(tweet.rawContent)

652473 At the same time it happened,
🔥Hyped #BTC    WL giveaway 🔥

2x @AntiNFTBTCMaxis
To Enter : 

1⃣Follow :@AntiNFTBTCMaxis , @D_C_famColab&amp;&amp; @d_cryptofamily 

2⃣ ♥️ , Retweet this post and paste your taproot address 

15 hrs 

#BTCHxRICH_Finale #btc #OrdinalsNFT #Ordinals https://t.co/r7rC7TQFYb
👇 I been holding on my wallet 233 ETHs, But this project $GPEPE will smoke pepe for sure. 🔋99 $PEPE $BOB $ROPE $LOYAL $PSALE $GMFAM $MONG $ETH $BTC $GPEPE $JESUS $BEN $RFD
👇 I just been airdrop $GPEPE as we speak, this one will do 1200X firsy day, calling it now. 🔋99 $PEPE $BOB $ROPE $LOYAL $PSALE $GMFAM $MONG $ETH $BTC $GPEPE $JESUS $BEN $RFD
Working on converting Bitcoin’s SHA256’s “padding” into 3D.
#Bitcoin #BTC https://t.co/tdNKKotraj
HAVING WITHDRAWAL ISSUE DM
#bstdcoin #bitlyx #zaifint #chainbil #batetiu #coinfusionx #exbills #hotbit #holbit #crypto #btc  #recovery #scam #invest #japan #china #turkey #xrp  #echovfx #india #vauld #amzdoge #cyberdoge #iran #ukraine #Temu #今月描いた

👇 Thank you so much for the airdrop, $GPEPE will do 700X tonight! 🔋99 $PEPE $BOB $ROPE $LOYAL $PSALE $GMFAM $MONG $ETH $BTC $GPEPE $JESUS $BEN $RFD
@JetherBlaise Used to differentiate LGBTQIA and woke BTC holders from the old BTC holders. Old ones use BTC , woke ones use XBT
Ahoy, degens! I just discovered a mysterious artifact on @Tabi_NFT.
Join me on the voyagers‘ expedition, where we uncover a wealth of exquisite treasures. Hop on board and let's catch the wind.  https://t.co/3xXuXJIThQ


@MavsFilmRoom OMP
@CryptoJoann @memecoin here
Nice zkSync Era∎ drop! Fck SEC. ✨ @kingwar_eth @repanpeisu47409 @cosmoakar @kreeksky
@bbykyoko @sochespilled whomst is this kyoko
Check out this prеsale faster, it's the next $PEPE 

@fortsatos14 @AdrianMot7 @maxtradercoltd @crypto_culinar @Olasquare104 @NawShine13 @kreativOff @PhatRooster @hit_map @magnificenticus @Aleksei02021985 @Win43017521994 
https://t.co/NJfbM0fqJe
@BoredApeYC First move is picking the proper playlist to Forge to 🤘
@GeordieChris_F @NUFCThreatLevel Tbh never ever notice him for England don't watch him week out for chelsea. Maybe eth sees something that we don't time will tell
@bayc1404 @BoredApeYC @EverydayZukini My set up
@Pauly0x Lol you’ve had all your vaccines I can tell by the way you project onto everyone else.
1940982😛😞toward perhaps of 
 #遵义 #湛江 #毕节  #韩城 #华阴 #兴平 https://t.co/FgAQWNJJdM


@reecefrailing 🙏
@Chels_HQ Who sold Mata to United?
Who sold cech to arsenal?
Matic? Find another agenda to spin
$BTC : 30542
$ETH : 1851.71
$ADA : 0.273662
$SOL : 17.67
$BNB : 233.4
$MATIC : 0.624327
$HIVE : 0.319477
$PVU : 0.00481674
$ZOON : 0.00050085
$SLP : 0.00167964
$AXS : 5.66
@katrinpetau 🙏
Thank you for the support @Dew_HQ 💜

Mucho Llamalove 🦙

#NFT #PFP #MATIC
Dreaming of a crypto bonanza? Make it happen at https://t.co/sSd2P2Mpos!
Join https://t.co/2RfmWF8caL

#BTC $ETH $ALGO #LTC #MATIC $MANA $SXP #ATOM https://t.co/6KJPRs77fc
@alejoUTD Donde está Juan Mata, Matic, William, David Luiz, Peter Chech, Lampard? United, Arsenal y City no ganaron nada con ellos.
Such abormination, why did she even agree to it in the first place and where are the rest of her family 🤦🏾‍♂️🤦🏾‍♂️ jah know
@BullishBrit the @Cre8orsNFT
🚄 #LINK and 54 Others,  🟢 29 🔴 26

🇰🇷 43.54억원
🇺🇸 $3,725,752
🇯🇵 ¥409,230,517
🇪🇺 €3,153,398
🇨🇳 24,052,437元

🎉Upflow volume Now, Upbit(Korea)
#LINK #BCH #BTC #MTL #SOL #EOS 

@HumpedUnicorn @Wrathtank_avax I hope you rot in hell
@DWOLFBTC Plot twist: they already have them
@faucet_trade send AVAX tokens to 0xBb42448294E1C8e450c1F7D18c70c49F38c51d83 
#avalanche_fuji_avax #avalanche_fuji #faucet #avax 
https://t.co/9OFKAxYWul
@nobsfud https://t.co/qDNx8RIBGw
WIP 🏗️ Preparing for tomorrows FAF on @0xCampfire …..
More piece to come 🤍🖤…

#alwAyzCLUELESS
#AVAX https://t.co/f5ZJNstXoK
@discoiinfernos @Wrathtank_avax hump you
@maislethevert @discoiinfernos @raffllrr @RIPDAOAVAX @cowncartel @ArtOnAvax @hodl_avax @Wrathtank_avax @SOPESGAL @virkkk_ @DanielxKilleen @mattxpalmieri 👀 https://t.co/RUVCt3oDG5
🚨 The $APE airdrop is here!

Read the rules on our site:
🔗 https://t.co/vd4Blc8Scy

#USDT #Ethereum #BAYC #NFTs #bitcoin $ARB $XRP #ETH #Binance Tether $PSYOP Matic #AVAX #crypto #APE #Coinbase $LTC $ADA $RNDR $BNB Polygon $FLOKI #BTC #100X #MAYC #APE Otherside $BTC #NFTSales
@MrBigWhaleREAL $ETH and $AVAX
@HumpedUnicorn @Wrathtank_avax Your humping unicorns are disgu

### Data Source 4/4 - Trading History

In [ ]:
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = "YOUR KEY"
# find your environment next to the api key in pinecone console
env = "us-west1-gcp-free"

pinecone.init(api_key=api_key, environment=env)
pinecone.whoami()

index_name = 'trading-bot'

import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pinecone.list_indexes():
    # if does not exist, create index
    pinecone.create_index(
        index_name,
        dimension=len(res['data'][0]['embedding']),
        metric='cosine'
    )
    # wait for index to be initialized
    while not pinecone.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pinecone.Index(index_name)
# view index stats
index.describe_index_stats()

query = f"Historical data: {crypto_data}\n News: {news_output} Twitter Data: {tweets_list}\n"

res = openai.Embedding.create(
    input=[query],
    engine="text-embedding-ada-002"
)

# retrieve from Pinecone
xq = res['data'][0]['embedding']

# get relevant contexts (including the questions)
res = index.query(xq, top_k=5, include_metadata=True)
print(res)

sum_of_scores = 0.0

# Get the list of matches
matches = res['matches']

biggest_id = 0
# Iterate over the list
for match in matches:
    # Add the score of the current match to the sum
      if int(match['id']) >= int(biggest_id):
        biggest_id = match['id']
        sum_of_scores += match['score']

# Calculate the average by dividing the sum by the number of matches
vector_delta = sum_of_scores / len(matches)

print("Vector Delta: ", vector_delta)
print("Biggest ID: ", biggest_id)

{'matches': [{'id': '5', 'score': 1.0, 'values': []},
             {'id': '1', 'score': 0.999998271, 'values': []},
             {'id': '6', 'score': 0.999998271, 'values': []},
             {'id': '3', 'score': 0.999998271, 'values': []},
             {'id': '2', 'score': 0.694373429, 'values': []}],
 'namespace': ''}
Vector Delta:  0.3999996542
Biggest ID:  6


# Step 3 - Prompt Engineering

In [ ]:
base_prompt = f"""
You are in control of my crypto trading profile. You should take into consideration the factors you have to determine the best trade. Here is the info:

You can execute these commands:

1. buy_crypto_price(symbol, amount)
2. buy_crypto_limit(symbol, amount, limit)
3. sell_crypto_price(symbol, amount)
4. sell_crypto_limit(symbol, amount, limit)
5. do_nothing()





Use this when you don't see any necessary changes.

You also have access to this data:

1. Historical data
2. News Headlines
3. Twitter Data
4. Vector Delta

Vector delta measures how similar the last iterations market environment was to this one. it's a number between 0 and 1, where 1 is the most similar.

The current date and time is {dt.datetime.today()}

You are called once every 30 minutes, keep this in mind.

The only cryptos you can trade are LINK, ETH, MATIC, SOL and LTC.

here are the data sources:


"""

# Convert the info into a format suitable for the AI prompt
info_str = f"Historical data: {crypto_data}\n News: {news_output} Twitter Data: {tweets_list}\n Vector Delta: {vector_delta}"
prompt = base_prompt + "\n\n" + info_str
user_prompt = """
What should we do to make the most amount of profit based on the info? Here are your options for a response.

1. buy_crypto_price(symbol, amount) This will buy the specified amount of the specified cryptocurrency.
2. buy_crypto_limit(symbol, amount, limit) This will set a limit order to buy the specified amount of the specified cryptocurrency if it reaches the specified limit.
3. sell_crypto_price(symbol, amount) This will sell the specified amount of the specified cryptocurrency.
4. sell_crypto_limit(symbol, amount, limit) This will set a limit order to sell the specified amount of the specified cryptocurrency if it reaches the specified limit.
5. do_nothing() Use this when you don't see any necessary changes.

Choose one.
CRITICAL: RESPOND IN ONLY THE ABOVE FORMAT. EXAMPLE: buy_crypto_price("ETHBTC", 0.1). DO NOT SAY ANYTHING ELSE.
ALSO IN THE AMOUNT FIELD, USE THE UNIT SYSTEM OF BITCOIN, NOT DOLLARS. ASSUME WE HAVE A BUDGET of UP TO $100 WORTH OF BITCOIN PER TRADE for 24 hours.
 ADD THE ACRONYM "BTC" AT THE END OF THE CRYPTO TICKER.
    """


openai.api_key = "YOUR KEY"

# Feed the prompt to the AI
response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature = 0.2,
    )

res = response.choices[0].message["content"]
res = res.replace("\\", "")
print(textwrap.fill(str(res), width=50))

sell_crypto_price("BTCUSD", 0.003)


Save Prediction

In [ ]:
prediction_embedding = openai.Embedding.create(
    input=[
        info_str
    ], engine="text-embedding-ada-002"
)
vector = prediction_embedding["data"][0]["embedding"]
pinecone_vectors = []


pinecone_vectors.append((str(int(biggest_id)+1), vector))
index.upsert(vectors=pinecone_vectors)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 6}},
 'total_vector_count': 6}

# Step 4 - Trade

In [ ]:
rest_api = tradeapi.REST('YOUR_KEY', 'YOUR_OTHER_KEY','https://paper-api.alpaca.markets')

def buy_crypto_price(symbol, amount):
  rest_api.submit_order(symbol=symbol, qty=amount , type="market", side="buy", time_in_force="gtc")

def buy_crypto_limit(symbol, amount, limit):
  rest_api.submit_order(symbol=symbol, qty=amount , type="limit", side="buy", time_in_force="gtc", limit_price=limit)

def sell_crypto_price(symbol, amount):
  rest_api.submit_order(symbol=symbol, qty=amount , type="market", side="sell", time_in_force="gtc")

def sell_crypto_limit(symbol, amount, limit):
  rest_api.submit_order(symbol=symbol, qty=amount , type="limit", side="sell", time_in_force="gtc", limit_price=limit)

In [ ]:
print(res)
def execute_response(response):
    match = re.match(r'(\w+)\((.*?)\)', response)
    if match:
        command = match.group(1)
        args = [arg.strip().strip('\"') for arg in match.group(2).split(',')]  # remove surrounding quotation marks
        if len(args) == 1:
          print("Doing nothing...")
          return
        command_map = {
            "buy_crypto_price": buy_crypto_price,
            "buy_crypto_limit": buy_crypto_limit,
            "sell_crypto_price": sell_crypto_price,
            "sell_crypto_limit": sell_crypto_limit,
            "do_nothing": lambda: None  # no action needed
        }
        function_to_execute = command_map.get(command)  # retrieves the function from command_map dictionary
        if function_to_execute:
            print(f"Executing command {function_to_execute} with args {args} in 5 seconds.")
            time.sleep(5)
            function_to_execute(*args)  # executes the function with its arguments
        else:
            print("Invalid command:", command)
    else:
        print("Invalid response, retrying:", response)
        time.sleep(10)
        execute_response(get_trade_advice())


execute_response(res)
print('done')

sell_crypto_price("BTCUSD", 0.003)
so here sell_crypto_price("BTCUSD", 0.003)
Executing command <function sell_crypto_price at 0x7f9c973869e0> with args ['BTCUSD', '0.003'] in 5 seconds.
done
